# Aggregate analysis walkthrough

Load the detection summary and produce quick sanity checks using pandas.

In [ ]:
from pathlib import Path

import pandas as pd

DATASET_ROOT = Path("..") / "data" / "datalad" / "bogus"
SUMMARY = DATASET_ROOT / "artifacts" / "aggregate" / "summary.csv"
summary_df = pd.read_csv(SUMMARY)
summary_df.head()

## Detections per label

In [ ]:
summary_df["label"].value_counts()

## Runtime vs confidence join (placeholder)

In [ ]:
telemetry_path = DATASET_ROOT / "data" / "telemetry" / "infer" / "log.jsonl"
print("Add join logic here once telemetry schema is finalized.")